# 8-дәріс: Белгілер инжинирингі және Логистикалық регрессия

**Дәріс мақсаттары:**
1. Белгілер инжинирингінің маңыздылығын және негізгі техникаларын түсіну.
2. Жетіспейтін деректермен, шығарындылармен және категориялық айнымалылармен жұмыс істеуді үйрену.
3. Сызықтық регрессияның классификация есептеріне неліктен сәйкес келмейтінін түсіну.
4. Логистикалық регрессияның математикалық негіздерін, соның ішінде логистикалық функцияны, мүмкіндіктерді (odds) және log-odds-ты зерттеу.
5. Классификация модельдерінің сапасын бағалаудың негізгі метрикаларын (Confusion Matrix, Accuracy, Precision, Recall, F1, ROC/AUC) меңгеру.
6. Логистикалық регрессияны бірнеше класс жағдайына кеңейтуді қарастыру.

## 1-бөлім: Белгілер инжинирингі (Feature Engineering)

**Белгілер инжинирингі (Feature Engineering)** — бұл пәндік сала туралы білімді пайдаланып, бар белгілерден жаңа белгілер жасау процесі. Мақсаты — машиналық оқыту алгоритмдері үшін негізгі мәселені ең жақсы сипаттайтын түрде деректерді ұсыну. Модельдің сапасы 80% алгоритмнің күрделілігіне емес, белгілердің сапасына байланысты.

#### Белгілер инжинирингіндегі үш негізгі тәсіл:

1. **Бөліп алу (Extraction):** Күрделі белгілерден жаңа, қарапайым белгілер жасау. Мысалы, толық уақыт белгісінен (`timestamp`) апта күнін, айды немесе тәулік уақытын бөліп алу.
2. **Біріктіру (Combination):** Бірнеше белгіні біреуге біріктіру. Мысалы, олардың бірлескен әсерін ескеру үшін полиномиалды белгілерді (`TV_budget * Radio_budget`) жасау.
3. **Түрлендіру (Transformation):** Бастапқы белгілердің қасиеттерін жақсарту үшін оларды өзгерту. Бұған масштабтау (`StandardScaler`), логарифмдеу немесе төменде қарастыратын категориялық белгілерді кодтау жатады.

### 1.1. Жоқ деректермен жұмыс (Missing Data)

Нақты деректер ешқашан толық болмайды. Бос орындар деректер жинау қателерінен, жүйелердің істен шығуынан немесе белгілі бір белгінің нысанға қолданылмайтындығынан (мысалы, гаражы жоқ үй үшін 'гараждың салынған жылы') пайда болуы мүмкін.

In [ ]:
import pandas as pd
import numpy as np

# Бос орындары бар қарапайым DataFrame құрайық
data = {'temperature': [25, 26, np.nan, 28, 29, 24],
        'humidity': [80, np.nan, 82, 83, 81, 79],
        'wind_speed': [10, 12, 11, np.nan, 13, 9]}
sample_df = pd.DataFrame(data)

print("Бос орындары бар бастапқы деректер:")
print(sample_df)

print("\n.isNone().sum() көмегімен бос орындарды іздеу:")
print(sample_df.isNone().sum())

#### 1.1.1. Деректерді жою

Ең қарапайым тәсіл. Жолдарды (`.dropna(axis=0)`) немесе бүкіл бағандарды (`.dropna(axis=1)`) жоюға болады. Бұл, егер бос орындар өте аз болса ғана ақталған, әйтпесе біз құнды ақпаратты жоғалтуымыз мүмкін.

In [ ]:
# Кез келген бос мәні бар жолдарды жою
print("Жолдарды жойғаннан кейінгі DataFrame:")
print(sample_df.dropna())

#### 1.1.2. Деректерді толтыру (импутация)

Неғұрлым тиімді әдіс. Бос орындарды толтыруға болады:
* **Қарапайым мәндермен:** нөлмен, орташа (`.mean()`), медиана (`.median()`) немесе модамен (ең жиі кездесетін мән).
* **Жетілдірілген әдістермен:** мысалы, басқа белгілер негізінде бос мәнді болжау арқылы.

In [ ]:
# Бос орындарды баған бойынша орташа мәнмен толтыру
print("Орташа мәнмен толтырғаннан кейінгі DataFrame:")
print(sample_df.fillna(sample_df.mean()))

### 1.2. Дубликаттармен жұмыс (Duplicates)

**Дубликаттар** — бұл деректер жинағындағы толығымен бірдей жолдар. Дубликаттардың болуы бірнеше мәселеге әкелуі мүмкін:

1. **Нәтижелердің бұрмалануы:** Дубликаттар белгілі бір бақылауларға негізсіз үлкен салмақ береді, бұл статистикалық көрсеткіштерді (орташа, медиана) ығыстырып, модельді оқытуға әсер етуі мүмкін.
2. **Деректердің жылыстауы (Data Leakage):** Ең күрделі мәселе. Егер бірдей жол оқыту және тест таңдамаларына түссе, модель тест деректерінде "тегін" дұрыс жауап алады. Бұл сапа метрикаларының жасанды түрде жоғарылауына және модельдің шынайы жаңа деректерде қалай жұмыс істейтіні туралы жалған түсінікке әкеледі.

Сондықтан деректерді талдаудың кез келген жобасындағы **бірінші қадам** толық дубликаттарды тексеру және жою болуы керек.

**Pandas-та олармен қалай жұмыс істеу керек:**
* **Анықтау:** `.duplicated().sum()` әдісі толық дубликаттардың санын тез санауға мүмкіндік береді.
* **Көру:** Барлық қайталанатын жолдарды (олардың "түпнұсқаларын" қоса) көру үшін `df[df.duplicated(keep=False)]` қолданылады.
* **Жою:** `.drop_duplicates()` әдісі дубликаттарды жояды, әдепкі бойынша әр жолдың бірінші кездесуін қалдырады.

In [ ]:
# Айқын дубликаттары бар DataFrame құрайық
data = {'Аты': ['Арман', 'Айгерім', 'Бауыржан', 'Арман', 'Әлия', 'Айгерім'],
        'Жасы': [25, 30, 35, 25, 28, 30],
        'Қала': ['Астана', 'Алматы', 'Шымкент', 'Астана', 'Қарағанды', 'Алматы']}
df_duplicates = pd.DataFrame(data)

print("Бастапқы DataFrame:")
print(df_duplicates)

# --- Дубликаттарды анықтау ---
print("\n--- Анықтау ---")
num_duplicates = df_duplicates.duplicated().sum()
print(f"Деректердегі толық дубликаттар саны: {num_duplicates}")

# Қайталанатын жолдардың өзін көрейік
print("\nБарлық қайталанатын жолдарды көрсету (түпнұсқаларды қоса):")
print(df_duplicates[df_duplicates.duplicated(keep=False)])


# --- Дубликаттарды жою ---
print("\n--- Жою ---")
df_cleaned = df_duplicates.drop_duplicates()
print("Дубликаттарды жойғаннан кейінгі DataFrame:")
print(df_cleaned)

# --- Тексеру ---
print("\n--- Тексеру ---")
print(f"Тазалаудан кейінгі дубликаттар саны: {df_cleaned.duplicated().sum()}")

### 1.3. Шығарындылармен жұмыс (Outliers)

**Шығарынды** — бұл басқаларынан қатты ерекшеленетін деректер нүктесі. Шығарындылар оқыту нәтижелерін бұрмалап, регрессия сызығын өздеріне "тартып" алуы мүмкін.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Қалыпты таралуды генерациялап, шығарындылар қосайық
np.random.seed(42)
data_normal = np.random.normal(loc=100, scale=10, size=100)
data_with_outliers = np.concatenate([data_normal, [180, 190, -50]]) # 3 шығарынды қосамыз
df_outliers = pd.DataFrame(data_with_outliers, columns=['value'])

# boxplot көмегімен визуализациялаймыз
plt.figure(figsize=(10, 5))
sns.boxplot(data=df_outliers, x='value')
plt.title('Box Plot көмегімен шығарындыларды анықтау')
plt.show()

#### Практикалық мысал: Квартиларалық ауқым (IQR) әдісі бойынша шығарындыларды шектеу

Бұл әдіс boxplot-тағы "мұртшалардың" статистикалық аналогы болып табылады. Шығарындылар деп мына шектерден тыс жатқан барлық нүктелер саналады:

<br>
$$ Төменгі\_шек = Q1 - 1.5 \cdot IQR $$
<br>
$$ Жоғарғы\_шек = Q3 + 1.5 \cdot IQR $$
<br>

мұндағы $Q1$ — 25-ші перцентиль, $Q3$ — 75-ші перцентиль, $IQR = Q3 - Q1$. Жоюдың орнына, біз мәндерді "шектей" аламыз, яғни шектен шығатын барлық мәндерді шекаралық мәндермен алмастырамыз.

In [ ]:
# Шектерді есептейміз
Q1 = df_outliers['value'].quantile(0.25)
Q3 = df_outliers['value'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"Төменгі шек: {lower_bound:.2f}")
print(f"Жоғарғы шек: {upper_bound:.2f}")

# Мәндерді шектейміз (clipping)
df_clipped = df_outliers.copy()
df_clipped['value'] = df_clipped['value'].clip(lower=lower_bound, upper=upper_bound)

# Нәтижені визуализациялаймыз
plt.figure(figsize=(10, 5))
sns.boxplot(data=df_clipped, x='value')
plt.title('Шығарындыларды шектегеннен кейінгі деректер')
plt.show()

### 1.4. Категориялық деректермен жұмыс

**Категориялық деректер** — бұл сандық мәндер емес, белгілерді қамтитын айнымалылар. Олар нысанның қандай да бір топқа жататындығын сипаттайды.

Екі негізгі түрі бар:
1. **Номиналды (Nominal):** Категориялардың ішкі реті жоқ. *Мысалдар: 'Қала' (Мәскеу, Қазан), 'Жыныс' (Ер, Әйел).*
2. **Реттік (Ordinal):** Категориялардың табиғи реті немесе дәрежесі бар. *Мысалдар: 'Өлшем' (S, M, L), 'Баға' (Нашар, Жақсы, Өте жақсы).*

Машиналық оқыту модельдерінің көпшілігі сандармен жұмыс істейді, сондықтан мәтіндік категорияларды түрлендіру қажет.

#### 1.4.0. Сандарды кодтау (Integer/Label Encoding)

Бұл түрлендірудің ең қарапайым тәсілі: әрбір бірегей категорияға бүтін сан (0, 1, 2 және т.б.) беріледі.

**Алайда, бұл әдісті өте сақтықпен қолдану керек!**

**Мәселе: Жалған реттілік**

Категорияларға `1`, `2`, `3` сандарын бергенде, машиналық оқыту модельдерінің көпшілігі оларды реттелген шамалар ретінде қабылдайды. Олар `3 > 2 > 1` деп "ойлайды".

* **Қашан жаман (номиналды деректер үшін):** Егер елдерді кодтасақ: `{'АҚШ': 1, 'Мексика': 2, 'Канада': 3}`, модель "Канада" "Мексикадан" қандай да бір мағынада "үлкен" немесе "маңызды" деген қате қорытынды жасауы мүмкін. Бұл деректерге модель сапасына зиян келтіруі мүмкін жалған ақпарат енгізеді.

* **Қашан жақсы (реттік деректер үшін):** Егер біздің категорияларымыздың табиғи, логикалық реті болса (мысалы, тағамның ащылық деңгейі), онда сандармен кодтау дұрыс таңдау болады. Бұл жағдайда `{'Mild': 1, 'Hot': 2, 'Fire': 3}` реті модель үшін пайдалы ақпарат береді.

**Қорытынды:** Сандармен кодтауды тек **реттік (ordinal)** белгілер үшін қолданыңыз. **Номиналды (nominal)** белгілер үшін бұл әдіс, әдетте, зиянды.

<table>
  <tr>
    <td><img src="https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec8-14.png" alt="Елдерді кодтау" width="400"></td>
    <td><img src="https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec8-16.png" alt="Ащылықты кодтау" width="400"></td>
  </tr>
</table>

#### 1.4.1. Үздіксіз деректерді категорияларға түрлендіру (Binning)

Кейде үздіксіз белгіні (мысалы, жас) категориялыққа айналдыру пайдалы. Бұл процесс **дискретизация** немесе **биннинг** (ағылш. *bin* — себет) деп аталады. Бұл модельге бастапқы деректерде байқай алмаған сызықтық емес тәуелділіктерді ұстауға көмектесуі мүмкін. Мысалы, жастың қандай да бір көрсеткішке әсері сызықтық емес болуы мүмкін: алдымен ол өседі, ал белгілі бір нүктеден кейін төмендей бастайды. Жас топтарын ('Жас', 'Ересек', 'Егде') құру модельге әр топқа өзінің жеке, тәуелсіз салмағын беруге мүмкіндік береді.

In [ ]:
# Әртүрлі жастағы серия құрайық
ages = pd.Series([15, 22, 35, 48, 65, 70, 28, 55])

# Жас топтары үшін шекараларды анықтаймыз (биндер)
# [0, 18) -> 0-17 жас
# [18, 35) -> 18-34 жас
# [35, 60) -> 35-59 жас
# [60, 100) -> 60-99 жас
bins = [0, 18, 35, 60, 100]

# Бұл топтар үшін атауларды анықтаймыз
labels = ['Бала', 'Жас', 'Ересек', 'Егде']

# Деректерді категорияларға бөлу үшін pd.cut функциясын қолданамыз
age_groups = pd.cut(ages, bins=bins, labels=labels, right=False)

print("Бастапқы жас:")
print(ages)
print("\nКатегорияларға бөлгеннен кейін (Binning):")
print(age_groups)

#### 1.4.2. Сандық кодтарды категорияларға түрлендіру (Type Conversion)

Нақты деректерде категориялық белгілер жиі сандармен кодталған болады. Мысалы, Ames Housing деректер жинағында `MSSubClass` (тұрғын үй түрі) белгісі 20, 30, 60 және т.б. сандармен берілген.

**Мәселе:** Егер бұл деректерді сол күйінде қалдырса, машиналық оқыту алгоритмдерінің көпшілігі (әсіресе сызықтық модельдер) оларды үздіксіз сандық айнымалылар ретінде қате түсіндіреді. Модель кластар арасында математикалық тәуелділік бар деп болжауы мүмкін (мысалы, 60 класы 20 класынан 3 есе 'үлкен' немесе 'маңызды'), бірақ шын мәнінде бұл жай ғана бірегей кодтар.

**Шешімі:** Мұны болдырмау үшін, Pandas-қа бұл бағанды категориялық ретінде қарастыру керектігін нақты көрсету қажет. Ең сенімді әдіс — One-Hot Encoding қолданбас **бұрын** осы бағанның деректер түрін `object` немесе `str`-ға түрлендіру. Осындай түрлендіруден кейін `pd.get_dummies` функциясы әр санды жеке категория ретінде дұрыс танып, оған өз бинарлы бағанын жасайды.

In [ ]:
# Мәселені имитациялайтын DataFrame құрайық
subclass_df = pd.DataFrame({'MSSubClass': [20, 30, 60, 20, 70]})

print("Бастапқы DataFrame және деректер түрі:")
print(subclass_df)
print(f"Бағанның деректер түрі: {subclass_df['MSSubClass'].dtype}")

# --- ДҰРЫС ТӘСІЛ ---

# 1-қадам: Деректер түрін 'object' (немесе 'str')-ға түрлендіреміз
subclass_df['MSSubClass'] = subclass_df['MSSubClass'].astype(str)

print("\nТүрлендіруден кейінгі деректер түрі:")
print(f"Бағанның жаңа деректер түрі: {subclass_df['MSSubClass'].dtype}")

# 2-қадам: Енді One-Hot Encoding қолданамыз
dummies = pd.get_dummies(subclass_df, drop_first=True)

print("\nДұрыс түрлендіруден кейінгі One-Hot Encoding нәтижесі:")
print(dummies)

#### 1.4.3. Номиналды белгілер үшін One-Hot Encoding

**Номиналды** белгілер үшін (реті жоқ жерде) **One-Hot Encoding** қолданылады. Ол әрбір категория үшін жаңа бинарлы баған (0 немесе 1) жасайды. Бұл осындай деректерді кодтаудың стандартты және ең қауіпсіз тәсілі.

![OHE](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec8-19.png)

In [ ]:
# Қалалармен мысал
cities = pd.Series(['Almaty', 'Astana', 'Almaty', 'Astana', 'Shymkent'])

print("Бастапқы деректер:")
print(cities)

print("\nOne-Hot Encoding-тен кейін (бірінші бағанды жою арқылы):")
# drop_first=True, фиктивті айнымалылар тұзағынан аулақ болу үшін
print(pd.get_dummies(cities, drop_first=True))

## 2-бөлім: Логистикалық регрессия

Нақты алгоритмді зерттемес бұрын, қандай есепті шешетінімізді нақты анықтап алайық.

### Классификация есебі

**Классификация (Classification)** — бұл мұғаліммен оқытудың негізгі есептерінің бірі. Оның мақсаты — нысанның алдын ала анықталған бірнеше **класстардың** (категориялардың) қайсысына жататынын болжау. Үздіксіз санды (мысалы, үй бағасын) болжайтын регрессия есебінен айырмашылығы, мұнда біз дискретті белгіні болжаймыз.

* **Регрессия "Қанша?" деген сұраққа жауап береді:** *Пәтер қанша тұрады? Ертең ауа температурасы қандай болады?*
* **Классификация "Қандай?" деген сұраққа жауап береді:** *Бұл хат — спам ба, жоқ па? Суретте қандай жануар бар?*

### Классификация есептерінің түрлері

Классификация есептері екі негізгі түрге бөлінеді:

1. **Бинарлы классификация (Binary Classification):**
 * **Бұл не:** Дәл екі өзара бірін-бірі жоққа шығаратын класс бар есеп. Бұл классификацияның ең кең таралған түрі.
 * **Мысалдар:**
 * Медициналық диагноз: ауру бар (`1`) немесе жоқ (`0`).
 * Спам-сүзгі: хат спам болып табылады (`1`) немесе жоқ (`0`).
 * Банктік скоринг: клиент несиені қайтарады (`1`) немесе қайтармайды (`0`).
 * Маркетинг: пайдаланушы жарнаманы басады (`1`) немесе баспайды (`0`).

2. **Көп класты классификация (Multiclass Classification):**
 * **Бұл не:** Екіден көп өзара бірін-бірі жоққа шығаратын класс бар есеп. Нысан олардың тек біреуіне ғана тиесілі бола алады.
 * **Мысалдар:**
 * Қолжазба сандарды тану: `0`, `1`, `2`, ..., `9`.
 * Жаңалықтарды тақырыптар бойынша жіктеу: 'Саясат', 'Спорт', 'Технологиялар', 'Мәдениет'.
 * Мәтіннің тоналдығын талдау: 'Позитивті', 'Бейтарап', 'Негативті'.
 * Ирис гүлдерін жіктеу (кейінірек көретініміздей): 'Setosa', 'Versicolor', 'Virginica'.

### Логистикалық регрессия: Бинарлы классификацияның негізгі құралы

**Бинарлы классификация** есептерін шешуге арналған негізгі және ең жиі қолданылатын алгоритм — **логистикалық регрессия**. Атауында "регрессия" сөзі болғанына қарамастан, бұл нысанның екі кластың біріне жату ықтималдығын болжайтын классификация әдісі болып табылады.

Оның негізгі мақсаты бинарлы классификация болғанымен, оны көп класты есептерді шешуге де кеңейтуге мүмкіндік беретін әдістер бар (мысалы, One-vs-Rest), мұны біз Фишердің Ирис деректер жинағы мысалында қарастырамыз.

### 2.1. Регрессиядан классификацияға

Бізде студенттер туралы деректер (дайындық сағаттары) және емтихан нәтижесі (тапсырды/тапсырмады) бар деп елестетейік. Сызықтық регрессия мұнда сәйкес келмейді, себебі оның болжамдары [0, 1] аралығынан шығып кетуі мүмкін.

![linear-to-logitic](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec8-37.png)

In [ ]:
from sklearn.linear_model import LinearRegression

# Синтетикалық деректер
hours = np.array([0.5, 0.75, 1, 1.25, 1.5, 1.75, 1.75, 2, 2.25, 2.5, 2.75, 3, 3.25, 3.5, 4, 4.25, 4.5, 4.75, 5, 5.5]).reshape(-1, 1)
passed = np.array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1])

lin_reg = LinearRegression()
lin_reg.fit(hours, passed)

plt.figure(figsize=(10, 6))
plt.scatter(hours, passed, color='blue', label='Деректер')
plt.plot(hours, lin_reg.predict(hours), color='red', label='Сызықтық регрессия')
plt.axhline(y=0, color='grey', linestyle='--')
plt.axhline(y=1, color='grey', linestyle='--')
plt.title('Неліктен сызықтық регрессия классификацияға сәйкес келмейді')
plt.xlabel('Дайындық сағаттары')
plt.ylabel('Нәтиже (0 - тапсырмады, 1 - тапсырды)')
plt.legend()
plt.show()

### 2.2. Логистикалық функция (Сигмоида)

Шешім — сызықтық модельдің шығысын S-тәрізді **сигмоида** арқылы өткізу, ол кез келген санды 0-ден 1-ге дейінгі ықтималдыққа түрлендіреді.

<br>
$$ \sigma(z) = \frac{1}{1 + e^{-z}} $$
<br>

In [ ]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

z = np.linspace(-10, 10, 100)
plt.figure(figsize=(10, 4))
plt.plot(z, sigmoid(z))
plt.title('Сигмоидалық функцияның графигі')
plt.xlabel('z')
plt.ylabel('$\sigma(z)$')
plt.grid(True)
plt.show()

Мысал: income = 1. Несиені қайтару ықтималдығы - 90%

![ExmplPossibility](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec8-41.png)

### 2.3. Логистикалық регрессия теңдеуі

Біз $z = w_0 + w_1x_1 + ...$ сызықтық бөлігін сигмоидаға қоямыз. Модель '1' класына жату ықтималдығын болжайды.

![Формула](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec8-48.png)

In [ ]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
log_reg.fit(hours, passed)

# График үшін тегіс қисық құрамыз
x_test = np.linspace(0, 6, 100).reshape(-1, 1)
y_prob = log_reg.predict_proba(x_test)[:, 1] # 1 класының ықтималдығы

plt.figure(figsize=(10, 6))
plt.scatter(hours, passed, color='blue', label='Деректер')
plt.plot(x_test, y_prob, color='green', label='Логистикалық регрессия')
plt.title('Классификация үшін логистикалық регрессия')
plt.xlabel('Дайындық сағаттары')
plt.ylabel('Емтиханды тапсыру ықтималдығы')
plt.legend()
plt.show()

2.3.1. Интерпретация: Ықтималдықтан Log-Odds-қа және керісінше

Сызықтық регрессиядан айырмашылығы, логистикалық регрессияның $w_i$ коэффициенттерін тікелей "$x_i$ бірлікке өзгергенде $y$-тің өзгеруі" деп түсіндіруге болмайды. Олар нәтижеге сигмоида арқылы сызықтық емес әсер етеді. Олардың мағынасын түсіну үшін **Мүмкіндіктер (Odds)** және **Мүмкіндіктер логарифмі (Log-Odds)** ұғымдарын енгізу керек.

**1-қадам: Ықтималдықтан Мүмкіндіктерге (Odds)**

Мүмкіндіктер — бұл оқиғаның болу ықтималдығының оның болмау ықтималдығына қатынасы. Егер емтиханды тапсыру ықтималдығы ($p$) 0.8-ге тең болса, онда тапсырмау ықтималдығы ($1-p$) 0.2-ге тең. Тапсыру мүмкіндіктері 0.8 / 0.2 = 4, немесе "4-тің 1-ге" қатынасы.

<br>
$$ Odds = \frac{p}{1 - p} $$
<br>


**2-қадам: Мүмкіндіктерден Мүмкіндіктер логарифміне (Log-Odds)**

Мүмкіндіктерден натуралды логарифм алсақ, біз $-\infty$-ден $+\infty$-ге дейін өзгеретін шаманы аламыз. Бұл — **Log-Odds**.

<br>
$$ Log\_Odds = \ln\left(\frac{p}{1 - p}\right) $$
<br>

**Логистикалық регрессияның негізгі идеясы:** Белгілердің $z = w_0 + w_1x_1 + ... + w_mx_m$ сызықтық комбинациясы шын мәнінде ықтималдықтың өзіне емес, **мүмкіндіктер логарифміне** арналған модель болып табылады.

<br>
$$ \ln\left(\frac{p}{1 - p}\right) = w_0 + w_1x_1 + ... + w_mx_m $$
<br>

Бұл $x_i$ белгісі бір бірлікке артқанда, **мүмкіндіктер логарифмі** $w_i$-ге артатынын білдіреді.


![logodds](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec8-71.png)

**3-қадам: Кері жол — Log-Odds-тан Ықтималдыққа**

Модельмен болжанған Log-Odds ($z$) арқылы ықтималдықты ($p$) қалай алуға болады? Жоғарыдағы теңдеуден $p$-ны өрнектейік:

1. Екі жағынан да экспонента аламыз: $$ \frac{p}{1 - p} = e^z $$
2. $(1-p)$-ға көбейтеміз: $$ p = e^z \cdot (1-p) $$
3. Жақшаларды ашамыз: $$ p = e^z - p \cdot e^z $$
4. $p$ бар барлық қосылғыштарды солға көшіреміз: $$ p + p \cdot e^z = e^z $$
5. $p$-ны жақша сыртына шығарамыз: $$ p(1 + e^z) = e^z $$
6. Соңғы формуланы аламыз: $$ p = \frac{e^z}{1 + e^z} $$

Егер алымын да, бөлімін де $e^z$-ке бөлсек, біз бұрыннан таныс сигмоида формуласын аламыз:

<br>
$$ p = \frac{e^z/e^z}{(1+e^z)/e^z} = \frac{1}{1/e^z + 1} = \frac{1}{1 + e^{-z}} = \sigma(z) $$
<br>

Осылайша, **сигмоида — бұл модельмен болжанған Log-Odds-тан ықтималдыққа кері өтудің математикалық тәсілі.**

### 2.4. Модельді оқыту: Максималды шындыққа жанасымдылық әдісі және Log Loss

Модель оңтайлы `w` салмақтарын қалай табады? Біз қателердің квадраттарының қосындысын (MSE) минимизациялаған сызықтық регрессиядан айырмашылығы, логистикалық регрессияда **Максималды шындыққа жанасымдылық әдісі (Maximum Likelihood Estimation, MLE)** қолданылады.

![whatisthebest](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec8-77.png)


**Қарапайым мысалмен интуиция:**
Сіз тиынды 10 рет лақтырып, 7 рет бүркіт, 3 рет жазу алдыңыз деп елестетіңіз. Сіздің ойыңызша, бүркіттің түсу ықтималдығы ($p$) қандай? Көпшілігі 0.7 деп айтады. Сіз интуитивті түрде MLE-ді қолдандыңыз.

**MLE логикасы:** Параметрдің ($p$) сондай мәнін табайық, онда **дәл біз бақылаған деректерді** алу ықтималдығы максималды болады.
* Егер $p=0.5$ (әділ тиын) болса, біздің тізбегіміздің ықтималдығы $(0.5)^7 \cdot (1-0.5)^3 \approx 0.00097$ болар еді.
* Егер $p=0.7$ болса, біздің тізбегіміздің ықтималдығы $(0.7)^7 \cdot (1-0.7)^3 \approx 0.00222$ болар еді.

$p=0.7$ болғанда ықтималдық жоғары. MLE — бұл ықтималдықты (шындыққа жанасымдылықты) максимизациялайтын $p$-ны табу процесі.

**Логистикалық регрессияға қатысты:** Алгоритм оқыту таңдамасының әрбір нысаны үшін модельдің оның шын белгісіне жақын ықтималдық (яғни '1' класы үшін 1-ге жақын және '0' класы үшін 0-ге жақын) беруінің шындыққа жанасымдылығын максимизациялайтын `w` салмақтарын таңдайды.

Математикалық тұрғыдан, шындыққа жанасымдылықты максимизациялау теріс логарифмдік шындыққа жанасымдылықты минимизациялауға эквивалентті. Логистикалық регрессия үшін бұл шығын функциясы **Log Loss** (немесе бинарлы кросс-энтропия) деп аталады.

Бұл функцияның **бір** нысан үшін көрінісі:

<br>
$$ Loss(y, \hat{p}) = -[y \cdot \log(\hat{p}) + (1 - y) \cdot \log(1 - \hat{p})] $$
<br>

мұндағы:
* $y$ — шын класс белгісі (0 немесе 1).
* $\hat{p}$ — модельмен болжанған, нысанның 1 класына жату ықтималдығы.

**Оның қалай жұмыс істейтінін түсінейік:**

* **1-жағдай: Шын белгі y = 1.**
  Формула $Loss(1, \hat{p}) = -[1 \cdot \log(\hat{p}) + (0) \cdot \log(1 - \hat{p})] = -\log(\hat{p})$ дейін жеңілдейді.
  Бұл шығынды минимизациялау үшін $-\log(\hat{p})$ мәні мүмкіндігінше аз болуы керек. Бұл $\log(\hat{p})$ максималды болғанда, яғни $\hat{p}$ **1**-ге ұмтылғанда орындалады. Бұл бізге дәл қажет нәрсе!

* **2-жағдай: Шын белгі y = 0.**
  Формула $Loss(0, \hat{p}) = -[0 \cdot \log(\hat{p}) + (1) \cdot \log(1 - \hat{p})] = -\log(1 - \hat{p})$ дейін жеңілдейді.
  Бұл шығынды минимизациялау үшін $-\log(1 - \hat{p})$ мәні мүмкіндігінше аз болуы керек. Бұл $\log(1 - \hat{p})$ максималды болғанда, яғни $1 - \hat{p}$ 1-ге ұмтылғанда, ал $\hat{p}$ өзі — **0**-ге ұмтылғанда орындалады. Тағы да, бұл бізге дәл қажет нәрсе!

N нысаннан тұратын бүкіл оқыту таңдамасы үшін жалпы **құн функциясын (Cost Function)** алу үшін біз бұл шығынды жай ғана орташалаймыз:

<br>
$$ J(w) = -\frac{1}{N} \sum_{i=1}^{N} [y_i \cdot \log(\hat{p}_i) + (1 - y_i) \cdot \log(1 - \hat{p}_i)] $$
<br>

Болжанған ықтималдық $\hat{p}_i$ сигмоидалық функцияны белгілердің сызықтық комбинациясына қолдану нәтижесі болғандықтан ($\hat{p}_i = \sigma(w^T x_i)$), минимизациялау қажет құн функциясының толық формуласы келесідей болады:

<br>
$$ J(w) = -\frac{1}{N} \sum_{i=1}^{N} \left[ y_i \cdot \log\left( \frac{1}{1 + e^{-w^T x_i}} \right) + (1 - y_i) \cdot \log\left(1 - \frac{1}{1 + e^{-w^T x_i}}\right) \right] $$
<br>

Бұл формула күрделі көрінгенімен, ол дөңес болып табылады, бұл жалғыз ғаламдық минимумның болуына кепілдік береді. Сызықтық регрессиядағыдай, осы минимумды (яғни оңтайлы `w` салмақтарын) табу үшін **градиенттік түсу әдісі** қолданылады.

![logodds-p](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec8-100.png)

### 2.5. Көп класты Логистикалық Регрессия

Егер бізде екіден көп класс болса не істеу керек? Стандартты тәсіл **One-vs-Rest (OvR) (Біреуі қалғандарына қарсы)** деп аталады. K класс үшін K бинарлы классификатор құрылады: біріншісі 1-класты қалғандарынан ажыратады, екіншісі — 2-класты қалғандарынан, және осылай жалғасады. Жаңа нысан үшін барлық K классификатор іске қосылады, және ең жоғары сенімділікті (ықтималдықты) бергені таңдалады.

Мұны классикалық **Фишердің Ирис** деректер жинағында қарастырайық.

In [ ]:
from sklearn.datasets import load_iris

iris = load_iris()
X_iris = iris.data
y_iris = iris.target

df_iris = pd.DataFrame(X_iris, columns=iris.feature_names)
df_iris['species'] = y_iris

# Көрнекілік үшін сандық белгілерді атаулармен алмастырамыз
target_names = iris.target_names
df_iris['species'] = df_iris['species'].apply(lambda x: target_names[x])

print("Фишердің Ирис деректер жинағының алғашқы 5 жолы:")
print(df_iris.head())

sns.pairplot(df_iris, hue='species')
plt.show()

`pairplot`-тан `setosa` класы оңай бөлінетіні, ал `versicolor` мен `virginica` ішінара қиылысатыны көрінеді. Барлық үш класты бөлу үшін модельді оқытайық.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

# Деректерді дайындау
X_train_iris, X_test_iris, y_train_iris, y_test_iris = train_test_split(X_iris, y_iris, test_size=0.3, random_state=42)

scaler_iris = StandardScaler()
X_train_iris_scaled = scaler_iris.fit_transform(X_train_iris)
X_test_iris_scaled = scaler_iris.transform(X_test_iris)

# Модельді оқыту. Scikit-learn көп класты есеп үшін OvR-ді автоматты түрде қолданады.
log_reg_multi = LogisticRegression()
log_reg_multi.fit(X_train_iris_scaled, y_train_iris)

# Бағалау
y_pred_iris = log_reg_multi.predict(X_test_iris_scaled)

print("Фишердің Иристері үшін қателіктер матрицасы:")
print(confusion_matrix(y_test_iris, y_pred_iris))
print("\nКлассификация есебі:")
print(classification_report(y_test_iris, y_pred_iris, target_names=target_names))

## 3-бөлім: Классификация модельдерін бағалау метрикалары

Регрессияға арналған метрикалар (MAE, RMSE) мұнда сәйкес келмейді. Классификация үшін **Қателіктер матрицасына (Confusion Matrix)** негізделген өз метрикалар жиынтығы қолданылады.

### 3.1. Қателіктер матрицасы (Confusion Matrix)

Бұл кесте модельдің әрбір класс үшін қанша болжамды дұрыс, қаншасын қате жасағанын көрсетеді. Ол барлық басқа метрикалардың негізі болып табылады.

* **True Positive (TP):** Ақиқат-оң. Модель '1' деп болжады, және ол шын мәнінде '1' болды.
* **True Negative (TN):** Ақиқат-теріс. Модель '0' деп болжады, және ол шын мәнінде '0' болды.
* **False Positive (FP):** Жалған-оң (I-түрдегі қате). Модель '1' деп болжады, бірақ шын мәнінде ол '0' болды.
* **False Negative (FN):** Жалған-теріс (II-түрдегі қате). Модель '0' деп болжады, бірақ шын мәнінде ол '1' болды.

![confusionmatrix](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec8-126.png)

In [ ]:
from sklearn.metrics import confusion_matrix

# Шамамен алынған шын мәндер мен болжамдар
y_true = [1, 0, 1, 1, 0, 1, 0, 0, 1, 0]
y_pred = [1, 1, 1, 0, 0, 1, 0, 0, 1, 0]

cm = confusion_matrix(y_true, y_pred)

sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Класс 0', 'Класс 1'], yticklabels=['Класс 0', 'Класс 1'])
plt.xlabel('Болжанған мәндер')
plt.ylabel('Шын мәндер')
plt.title('Қателіктер матрицасы')
plt.show()

### 3.2. Негізгі метрикалар

#### **Accuracy (Дұрыс жауаптар үлесі)**
* **Сұрақ:** Жалпы бақылаулар санындағы барлық дұрыс болжамдардың (оң және теріс) үлесі қандай?
* **Формула:** $$ Accuracy = \frac{TP+TN}{TP+TN+FP+FN} $$
* **Диапазон:** 0-ден 1-ге дейін (немесе 0%-дан 100%-ға дейін).
 * **1-ге жақын:** Модель жалпы алғанда көптеген дұрыс болжамдар жасайды.
 * **0-ге жақын:** Модель жиі қателеседі.
* **Интерпретация:** Қарапайым және түсінікті метрика. Алайда, ол **теңгерімсіз деректерде** — бір кластың екіншісінен әлдеқайда көп болған жағдайларда адастыруы мүмкін. Бұл құбылыс **Дәлдік парадоксы** деп аталады.

**Дәлдік парадоксының мысалы:**
Сирек кездесетін ауруды диагностикалау есебін елестетейік. 1000 адамдық таңдамада: 990-ы сау (0 класы) және 10-ы ауру (1 класы). Біз әрқашан "сау" (0 класы) деп болжайтын өте қарапайым (және пайдасыз) модель құрдық.
* **TN:** 990 (сау адамдарды дұрыс тапты).
* **TP:** 0 (бірде-бір ауру адамды таппады).
* **FP:** 0 (ешкімді ауру деп атамады).
* **FN:** 10 (барлық ауру адамдарды өткізіп алды).

Оның Accuracy-і: $$ Accuracy = \frac{0 + 990}{0 + 990 + 0 + 10} = \frac{990}{1000} = 99\\% $$. 
99% дәлдік керемет болып көрінеді, бірақ модель мүлдем пайдасыз, себебі ол басты міндетті — ауруларды табуды — шешпейді. Сондықтан теңгерімсіз деректер үшін `Accuracy`-ді қолдануға болмайды. Басқа метрикалар қажет.

![Precision1](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec8-134.png)

![Precision2](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec8-137.png)

#### **Recall (Толықтық, немесе Сезімталдық)**
* **Сұрақ:** Барлық нақты 'оң' нысандардың ішінен біз қандай үлесін таба алдық?
* **Формула:** $$ Recall = \frac{TP}{TP + FN} $$
* **Диапазон:** 0-ден 1-ге дейін.
 * **1-ге жақын:** Модель оң кластың барлық дерлік нысандарын табады.
 * **0-ге жақын:** Модель оң кластың көптеген нысандарын өткізіп жібереді.
* **Интерпретация:** Recall **False Negative** қатесінің құны жоғары болғанда маңызды. *Мысал: медициналық диагностика. Ауру адамды өткізіп алу (FN) — апат. Біз модельдің мүмкіндігінше көп нақты ауруларды тапқанын қалаймыз, тіпті егер ол кейде сау адамдарды қосымша тексеруге (FP) қате жіберіп отырса да. Мұнда жоғары Recall маңызды.* 

![Recall1](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec8-146.png)

![Recall2](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec8-148.png)

#### **Precision (Дәлдік)**
* **Сұрақ:** Модель 'оң' деп атаған барлық нысандардың ішінен қандай үлесі шын мәнінде 'оң' болды?
* **Формула:** $$ Precision = \frac{TP}{TP + FP} $$
* **Диапазон:** 0-ден 1-ге дейін.
 * **1-ге жақын:** Модель өзінің оң болжамдарында өте дәл. Егер ол "Иә" десе, оған сенуге болады.
 * **0-ге жақын:** Модельдің оң болжамдарының көпшілігі — жалған дабылдар.
* **Интерпретация:** Precision **False Positive** қатесінің құны жоғары болғанда маңызды. *Мысал: спам-сүзгі. Егер модель маңызды хатты спам деп белгілесе (FP), бұл үлкен мәселе. Біз модельдің өзінің "спам"-үкімдеріне өте сенімді болғанын, яғни жоғары Precision-ге ие болғанын қалаймыз.* 

![Precision1](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec8-150.png)

![Precision2](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec8-152.png)


#### **F1-Score**
* **Сұрақ:** Precision мен Recall арасындағы тепе-теңдікті қалай табуға болады?
* **Формула:** $$ F1 = 2 \cdot \frac{Precision \cdot Recall}{Precision + Recall} $$
* **Диапазон:** 0-ден 1-ге дейін.
 * **1-ге жақын:** Модельде дәлдік пен толықтық арасында жақсы тепе-теңдік бар.
 * **0-ге жақын:** Модельдің не дәлдігі, не толықтығы, немесе екеуі де төмен.
* **Интерпретация:** Precision мен Recall арасындағы гармоникалық орташа. Бұл метрика екі қателік те (FP және FN) маңызды болғанда және олардың арасында ымыра табу қажет болғанда пайдалы. Ол метрикалардың бірі (Precision немесе Recall) өте төмен болған модельдерді жазалайды.

In [ ]:
from sklearn.metrics import classification_report

# y_true and y_pred are defined in the cell with id: 35da37fa
print(classification_report(y_true, y_pred))

### 3.3. ROC-қисығы және AUC

Логистикалық регрессия ықтималдықтарды береді. Класстарды (0 немесе 1) алу үшін біз шекті (әдетте 0.5) қолданамыз. **ROC-қисығы** модельдің *барлық мүмкін* шекті мәндердегі сапасын көрсетеді, True Positive Rate (Recall) мен False Positive Rate арасындағы тәуелділік графигін құрады.

**AUC (Қисық астындағы аудан)** — бұл шектен тәуелсіз, интегралды сапа метрикасы.
* AUC = 1.0 — идеалды классификатор.
* AUC = 0.5 — кездейсоқ болжау.

Идеалды ROC & Шынайы ROC

<table>
  <tr>
    <td><img src="https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec8-177.png" alt="Идеалды" width="400"></td>
    <td><img src="https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec8-162.png" alt="Шынайы" width="400"></td>
  </tr>
</table>

In [ ]:
from sklearn.metrics import RocCurveDisplay

# log_reg, hours, passed айнымалылары 675442e8 және a5ade196-c76f-40ef-9096-b865796133b5 ұяшықтарында анықталған.
RocCurveDisplay.from_estimator(log_reg, hours, passed)
plt.plot([0, 1], [0, 1], 'k--', label='Кездейсоқ болжау (AUC = 0.5)')
plt.title('ROC-қисығы')
plt.legend()
plt.show()